In [1]:
#!pip install simple_salesforce

In [2]:
import json
from sqlalchemy import create_engine
import pandas as pd
import pymysql
pymysql.install_as_MySQLdb()

In [3]:
# import sys
# sys.path.append('../../../../')
from config import sf_username, sf_password, sf_security_token
from config import remote_db_endpoint, remote_db_port
from config import remote_db_name, remote_db_user, remote_db_pwd

In [4]:
from simple_salesforce import Salesforce
sf = Salesforce(username=sf_username, password=sf_password, security_token=sf_security_token)

In [5]:
engine = create_engine(f"mysql://{remote_db_user}:{remote_db_pwd}@{remote_db_endpoint}:{remote_db_port}/{remote_db_name}")
conn = engine.connect()

In [6]:
class_data_df = pd.read_sql("SELECT * FROM class", conn)
class_data_df.head(30)

,ID_Class,ID_Course,Section,StartDate,EndDate
0,1,1,GWU-ARL-DATA-PT-09-0,2020-03-16,2020-03-14
1,2,1,GWDC201805DATA3,2018-05-15,2018-11-08
2,3,2,GWARL201905WEB3,2019-05-14,2019-11-07
3,4,3,GWARL201905UIUX3,2019-05-14,2019-11-07


In [7]:
class_data_df.rename(columns={
    'ID_Class':'ID_Class__c',
    'ID_Course':'ID_Course__c',
    'Section':'Section__c',
    'StartDate':'Start_Date__c',
    'EndDate':'End_Date__c',    
}, inplace=True)
class_data_df.head()

,ID_Class__c,ID_Course__c,Section__c,Start_Date__c,End_Date__c
0,1,1,GWU-ARL-DATA-PT-09-0,2020-03-16,2020-03-14
1,2,1,GWDC201805DATA3,2018-05-15,2018-11-08
2,3,2,GWARL201905WEB3,2019-05-14,2019-11-07
3,4,3,GWARL201905UIUX3,2019-05-14,2019-11-07


In [8]:
class_data_df = class_data_df[['ID_Class__c', 'ID_Course__c', 'Section__c', 'Start_Date__c', 'End_Date__c']]
class_data_df.head()

,ID_Class__c,ID_Course__c,Section__c,Start_Date__c,End_Date__c
0,1,1,GWU-ARL-DATA-PT-09-0,2020-03-16,2020-03-14
1,2,1,GWDC201805DATA3,2018-05-15,2018-11-08
2,3,2,GWARL201905WEB3,2019-05-14,2019-11-07
3,4,3,GWARL201905UIUX3,2019-05-14,2019-11-07


In [9]:
class_data_records = class_data_df.to_dict('records')
class_data_records

[{'ID_Class__c': 1,
  'ID_Course__c': 1,
  'Section__c': 'GWU-ARL-DATA-PT-09-0',
  'Start_Date__c': datetime.date(2020, 3, 16),
  'End_Date__c': datetime.date(2020, 3, 14)},
 {'ID_Class__c': 2,
  'ID_Course__c': 1,
  'Section__c': 'GWDC201805DATA3',
  'Start_Date__c': datetime.date(2018, 5, 15),
  'End_Date__c': datetime.date(2018, 11, 8)},
 {'ID_Class__c': 3,
  'ID_Course__c': 2,
  'Section__c': 'GWARL201905WEB3',
  'Start_Date__c': datetime.date(2019, 5, 14),
  'End_Date__c': datetime.date(2019, 11, 7)},
 {'ID_Class__c': 4,
  'ID_Course__c': 3,
  'Section__c': 'GWARL201905UIUX3',
  'Start_Date__c': datetime.date(2019, 5, 14),
  'End_Date__c': datetime.date(2019, 11, 7)}]

In [10]:
for rec in class_data_records:
    
    record = {
        'ID_Class__c': rec['ID_Class__c'],
        'ID_Course__c': rec['ID_Course__c'],
        'Section__c': rec['Section__c'],
        'StartDate__c': str(rec['Start_Date__c']),
        'EndDate__c': str(rec['End_Date__c']),
    }
       
    try:
        sf.Class__C.create(record)
    except Exception as e:
        print(e)

Malformed request https://na111.salesforce.com/services/data/v42.0/sobjects/Class__C/. Response content: [{'message': "No such column 'ID_Class__c' on sobject of type Class__c", 'errorCode': 'INVALID_FIELD'}]
Malformed request https://na111.salesforce.com/services/data/v42.0/sobjects/Class__C/. Response content: [{'message': "No such column 'ID_Class__c' on sobject of type Class__c", 'errorCode': 'INVALID_FIELD'}]
Malformed request https://na111.salesforce.com/services/data/v42.0/sobjects/Class__C/. Response content: [{'message': "No such column 'ID_Class__c' on sobject of type Class__c", 'errorCode': 'INVALID_FIELD'}]
Malformed request https://na111.salesforce.com/services/data/v42.0/sobjects/Class__C/. Response content: [{'message': "No such column 'ID_Class__c' on sobject of type Class__c", 'errorCode': 'INVALID_FIELD'}]


In [11]:
class_lookup_list = []

# The `Name` column in the primary key in Salesforce objects
data = sf.query_all_iter("SELECT ID_Course__c, Name FROM Class__c")
for row in data:
    rec = {
        'Class__c': row['Name'],
        'ID_Course__c': row['ID_Course__c']
    }
    class_lookup_list.append(rec)

SalesforceMalformedRequest: Malformed request https://na111.salesforce.com/services/data/v42.0/query/?q=SELECT+ID_Course__c%2C+Name+FROM+Class__c. Response content: [{'message': "\nSELECT ID_Course__c, Name FROM Class__c\n       ^\nERROR at Row:1:Column:8\nNo such column 'ID_Course__c' on entity 'Class__c'. If you are attempting to use a custom field, be sure to append the '__c' after the custom field name. Please reference your WSDL or the describe call for the appropriate names.", 'errorCode': 'INVALID_FIELD'}]

In [12]:
class_lookup_df = pd.DataFrame(class_lookup_list)
class_lookup_df

""


In [ ]:
query = '''
    SELECT
        c.*
        ,co.CourseCode
    FROM
        class c
        INNER JOIN course co
        ON c.ID_Course = co.ID_Course
'''
class_data_df = pd.read_sql(query, conn)
class_data_df.head()

In [ ]:
course_data_df.rename(columns={
    'ID_Course':'Name',
    'CourseCode':'Course_Code__c',
    'CourseName':'Course_Name__c',
    'CreditHours':'Credit_Hours__c',
    'BootCampCourse':'Boot_Camp_Course__c',    
}, inplace=True)
course_data_df.head()

In [ ]:
try:
    sf.bulk.Lead.insert(student_load)
except Exception as e:
    print(e)